In [3]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import re

In [4]:
PATH_BACC = os.path.abspath('/Users/iminhas/work/UKE/februar_2022/data/Testfile_23.05.22.xlsx')
PATH_MERGE_Test = os.path.abspath('/Users/iminhas/work/UKE/februar_2022/data/BACC_Merge_23.05.22.xlsx')

bacc_df = pd.read_excel(PATH_BACC,'Tabelle1')
merge_df = pd.read_excel(PATH_MERGE_Test,'Tabelle1')

In [5]:
class VariableMapping():
    
    def __init__(self,file_dir:str,sheet):
        ''' to init file: File with new & old variablename, type & coding
        '''
        self.mapping = None
    
        ## check if file exists
        if not os.path.abspath(file_dir):
            raise FileNotFoundError('Input file' + file_dir + 'does not exist!')
    
        mapping = pd.read_excel(file_dir, sheet_name=sheet)

    
        ## ready:
        self.mapping = mapping  
        
        
        
    def value_to_string(self,value):
        
        if (value == -1111) | (value == -9999):
            return ''
        else:
            return value
        
        
    def append_new_variable_col(self,df:pd.DataFrame):
        ''' appends new variable from vmdf in df '''
        # Was mache ich für den Fall, wenn mehrere Variablen Namen auf einen neuen verweisen?
        # Es werden  3 Spalten mit den selben Namen erzzeugt.Beispiel für neue variable ['tx_med_p2y12']

    
        matches = list(set(df.columns) & set((self.mapping.Kurzname)) )
        matches_df = self.mapping.loc[self.mapping['Kurzname'].isin(matches)]
        
        ### List with new Variable Name
        new_var_list = matches_df['Field_Name'].to_list()
        ### Delete all Duplicates 
        clean_new_var_list = pd.unique(new_var_list)
 
        return pd.concat([df,pd.DataFrame(columns=clean_new_var_list)])
        
    
    def append_value_list(self,df:pd.DataFrame, patientID :str):
        '''insert the contet of old_col_name in new_col_name '''
        
        append_value_without_recode = df.loc[(self.mapping['Recode'] == 0)]
        #['Kurzname', 'Field_Name']
        
        return append_value_without_recode
    
 
    
    def check_dtype_timestamp(self,df:pd.DataFrame,col_name:str):
        ''' checks if dtype of cols are in right format'''
         
        # check if dtype is datetime
        
        
        if not np.issubdtype(df[col_name].dtype, np.datetime64) == True:
            raise ValueError(f"Incorrect data format f{col_name}, should be dtype datetime64")
            
    
    def correct_timestamp(self, date_col:pd.Series):
        
        for date in date_col:
            if not dt.datetime.strptime(date,'%Y-%m-%d'):
                raise ValueError("Incorrect data format, should be YYYY-MM-DD")
    


    
    def datetime(self,df:pd.DataFrame):
        ''' Create a column with datatime '''
        # ToDo: wenn Jahr nicht DDMMYYYY ist ändern und dann übernehmen, falls ja direkt übernehmen.
        # ToDo: Check if value 
        # Name: fi_adm0062, dtype: datetime64[ns]
        
        ### change datetime from ymd to dmy
        ## select all with 'transformation' == 'DDMMYYY' as df 
        datetime_dmy_df = self.mapping.loc[self.mapping['timestamp_code'] == 'DDMMYYY'] 
        
        #x = dt.datetime.strptime('2001-04-19','%Y-%m-%d').strftime('%d.%m.%Y')
        #print(x)
        
        ### filters all 'Field_Name' with timestamp = 1(Date with YYYY) in a set
        timestamp_yyyy = set(self.mapping.loc[(self.mapping.timestamp==2)]['Field_Name'])
        ## create a df with value from timestamp_yyyy
        timestamp_yyyy_df = self.mapping.loc[self.mapping['Field_Name'].isin(timestamp_yyyy)]
        ## create a column with 'Field_Name' and fill the date with YYYY
        for variable_new in timestamp_yyyy_df['Field_Name']:
            # get the old variable name
            variable_index_old = timestamp_yyyy_df.iloc[[timestamp_yyyy_df['Field_Name'].isin([variable_new])]]['Kurzname']
            variable_old = list(variable_index_old)[0]
            # create column with new variable name with new value of old variable name
            df[variable_new] = pd.DatetimeIndex(df[variable_old], yearfirst=True).year
            
        ### filters all 'Field_Name' with timestamp = 2(Date with MM) in a set
        timestamp_mm = set(self.mapping.loc[(self.mapping.timestamp==3)]['Field_Name'])
        ## create a df with value from timestamp_mm
        timestamp_mm_df = self.mapping.loc[self.mapping['Field_Name'].isin(timestamp_mm)]
        ## create a column with 'Field_Name' and fill the date with MM
        for variable_new in timestamp_mm_df['Field_Name']:
            # get the old variable name
            variable_index_old = timestamp_mm_df.iloc[[timestamp_mm_df['Field_Name'].isin([variable_new])]]['Kurzname']
            variable_old = list(variable_index_old)[0]
            # create column with new variable name with new value of old variable name
            df[variable_new] = pd.DatetimeIndex(df[variable_old], yearfirst=True).month
            
        ### filters all 'Field_Name' with timestamp = 4 & 'Typ'= timestamp (in a set)
        timestamp_col_merge = set(self.mapping.loc[((self.mapping.timestamp==4) & (self.mapping['Typ'] == 'timestamp'))]['Field_Name'])
        ## create a df with value from timestamp_mm
        timestamp_col_merge_df = self.mapping.loc[self.mapping['Field_Name'].isin(timestamp_col_merge)]
        
        ## bring to the right format
        #de_com005 und de_com007
        
        timestamp_dmy_df = timestamp_col_merge_df.loc[(timestamp_col_merge_df.timestamp_code=='DDMMYYYY')]
        timestamp_ymd_df = timestamp_col_merge_df.loc[(timestamp_col_merge_df.timestamp_code=='YYYYMMDD')]
        
        
        ## takes the value and merge them in one new var 
        for value in timestamp_col_merge:
            timestamp_value_1_df = self.mapping.loc[(self.mapping['Field_Name']==value) & (self.mapping['col_merge_1']==1)]
            timestamp_value_2_df = self.mapping.loc[(self.mapping['Field_Name']==value) & (self.mapping['col_merge_2']==1)]
                        
            
            ## select old var_name of all col
            # Kann ich den Inhalt besser entnehmen? Hier entnehme ich den ersten Inhalt einer Liste 
            timestamp_var_old_1 = list(timestamp_value_1_df['Kurzname'])[0]
            timestamp_var_old_2 = list(timestamp_value_2_df['Kurzname'])[0]
  
            
            ## create a col with new var name
            
            df[timestamp_var_old_1] = df[timestamp_var_old_1].apply(self.convert_to_DMY)

            #df[value] = pd.to_datetime(df[timestamp_var_old_1].astype(str) +' '+ df[timestamp_var_old_2].astype(str)) 
            df[value] = pd.to_datetime(df[timestamp_var_old_1] +' '+ df[timestamp_var_old_2].astype(str)) 
            print(value)
            #df['bacc_cpudecision']

            return df
    
    def convert_to_DMY(self,date):
        
        print(date)
        
        #dmy = dt.datetime.strptime(date,'%d.%m.%Y') #.strftime('%d.%m.%Y') 
        
        #print(dmy)


        #dmy = dt.datetime.strptime(date,'%Y-%m-%d') #.strftime('%d.%m.%Y') 
        dmy = date.strftime('%d.%m.%Y') #.strptime(date,'%Y-%m-%d')
        
        print(dmy)
        return dmy
    
    def categorical(self,df:pd.DataFrame):
        
        ### filters all 'Field_Name' with Typ = categorical in a set
        categorical_oldvar_set = set(self.mapping.loc[(self.mapping.Typ=='categorical')]['Field_Name'])  
        
        
                             
    def no_recode(self,df:pd.DataFrame):
        
        no_recode_var = set(self.mapping.loc[(self.mapping.recode == 0)]['Field_Name'])
        no_recode_df = self.mapping.loc[self.mapping['Field_Name'].isin(no_recode_var)]

    
        for new_var in no_recode_df['Field_Name']:
            print(new_var)
            old_var = no_recode_df.loc[list(no_recode_df['Field_Name']== new_var)]['Kurzname']
            old_var_1 = list(old_var)[0]
            if old_var_1 in df.columns:
                df[new_var] = df[old_var_1].apply(self.value_to_string)
            
            
        return df


        

In [6]:
bacc = VariableMapping(PATH_MERGE_Test,'Tabelle1')